In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
!pip install pymongo

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/') 
db = client['image_similarity_db_final']
collection = db['similarity_data_final']

In [3]:
def insert_data_to_mongodb(data):

    try:
        # Convert score values from string to float
    
        data['global_contrast_sim'] = float(data['global_contrast_sim'])
        data['local_contrast_sim'] = float(data['local_contrast_sim'])
        data['label_sim'] = float(data['label_sim'])
        data['texture_sim'] = float(data['texture_sim'])
        data['restnet_sim'] = float(data['restnet_sim'])
        data['aggregated_sim'] = float(data['aggregated_sim'])

        result = collection.insert_one(data)
        print(f"Data inserted with ID: {result.inserted_id}")
    except Exception as e:
        print(f"Error while inserting data: {str(e)}")

In [ ]:
# data = {
#     'img_name': 'apple.jpg',
#     'initial_image': '{base64 encoded image}',
#     'generated_image': '{base64 encoded image}',
#     'global_contrast_sim': '1.0',
#     'local_contrast_sim': '1.0',
#     'label_sim': '1.0',
#     'texture_sim': '1.0',
#     'restnet_sim': '0.875225305557251',
#     'aggregated_sim': '0.9750450611114503'
#      'model': 'model_name'
# }

# insert_data_to_mongodb(data)

In [4]:
from pyngrok import ngrok

In [ ]:
!E:\ngrok\ngrok-v3-stable-windows-amd64\ngrok.exe authtoken NGROK_AUTHTOKEN

Authtoken saved to configuration file: C:\Users\adity\AppData\Local/ngrok/ngrok.yml


In [ ]:
# !E:\ngrok\ngrok-v3-stable-windows-amd64\ngrok.exe authtoken NGROK_AUTHTOKEN

Authtoken saved to configuration file: C:\Users\adity\AppData\Local/ngrok/ngrok.yml


In [6]:
from typing import Dict
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import base64
from PIL import Image
import io

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class Data(BaseModel):
    data: Dict[str,str]

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/receive")
def receive(query: Data):
    r_data = query.data
    # Here you would call your text-to-image model, for example:
    # image = generate_image_from_model(prompt)
    data = r_data
    for key, value in data.items():
        print(f"Key: {key}, Value: {value}")
    
    insert_data_to_mongodb(data)
    
    # For the sake of an example, we'll generate a simple image
    # image = Image.new('RGB', (100, 100), color = 'blue')

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# ngrok_tunnel = ngrok.connect(8001, domain="useful-normal-giraffe.ngrok-free.app")
ngrok_tunnel = ngrok.connect(8001, domain="jointly-amazed-clam.ngrok-free.app")
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8001)

INFO:     Started server process [9196]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


Public URL: https://jointly-amazed-clam.ngrok-free.app
Key: img_name, Value: apple.jpg
Key: initial_image, Value: UklGRgQYAABXRUJQVlA4IPgXAADQrQCdASpYApABPlEkkEajoaGrIfBYsWAKCWdu9JfTp20ymZ/tWnqJ4qxJkQ3OeecP5wG/VU79+yuiMNNoID0K1+QImvRBB6wrX4NbP5AkBICPk6F54vceUXwXXTCb1tl8X9GRySHrkjznZXKYAoQI7s4kZXrkPx0QH9Du+W1DmunwUltzdXt0tDf4ratqOKO/yfhX78+BOfhckrCAd7mtECV8J0dkl69bNZ4YeQ8p2zL2sT+oX56tWXUW8YIE1YcTKaYZqcAzvTrRI5MjgiplsQzuLnO9fzoHyZ/fGyVzEEmUH1ITLYPYbRBB3BojhEJegKNkRfVakD5shtv8tV1BqxLoPVYchknYkfngAkaXW00Yw1tmArukFuGX/ejsCLNEagikcCCT4FuLlfn2VzaSPlzr4zFD2vUWUD/EltWVGEbjADNWbhOgb+TZEVSLEkhjnLtMhXJFQDgz7ZsZr9UdVhano9Wyv5uX7Krq11sbNsHTuMltx6NSj1Q/LNyjWlSFcoanXV37m+XxkrHZ//MKms2GSrsI03k8krwbTVb/AVr4lbJQtXYepTC/NgRy208PWWV3qyS1cccKycMExRcy6fIH0UguTCDbM2VEqa+O2K+MlLVJ/bEnA+1wp3EuDipFgO2P/hxSqKtnpKYQ7J0C93O6X7yFpAMSAkCBfHSV1Z2VLDsBwQ1hfHoiDeAG51jObEFhiBQOLNQvO0ewKX2v9h5aSVNOg6osBpe/m7T811un3v/RgsJFI6QBbOSm9cGtqVHS6UIjDz5uB1vPkCQM+V5oQLClzhBKZVNZwMBAJ52tqRv8ZLZwdEGj2UneHw0dYdNAl97+ahJuBL7RtE7ReEi2pT